In [1]:
!pip install torch_geometric
!pip install captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.7 MB/s eta 0:00:00


In [2]:
from google.colab import files
files.upload()

#jupytext --to ipynb Comparative_Cheb_GCN.py
# Don't forget to upload model as well
!unzip numpy_data.zip

Saving numpy_data.zip to numpy_data.zip
Saving model_34.pt to model_34.pt
Archive:  numpy_data.zip
  inflating: AdjacencyMatrix.npy     
  inflating: new_data.npy            
  inflating: new_labels.npy          
  inflating: NodeIndex.npy           


In [ ]:
import time
start_time = time.time()

#
#!pip install torch_geometric
#!pip install pdf2image
#!apt-get install poppler-utils
## !pip install captum
#
#from google.colab import files
#files.upload()
#
#jupytext --to ipynb Comparative_Cheb_GCN.py
# Don't forget to upload model.pth as well
#!unzip numpy_data.zip

from torch_geometric.data import Data, DataLoader
from torch_geometric.datasets import TUDataset, Planetoid
from torch_geometric.nn import GCNConv, Set2Set
from torch_geometric.explain import GNNExplainer
import torch_geometric.transforms as T
import torch
#import torch.nn.functional as F
import os
from tqdm import tqdm, trange
#
import matplotlib.pyplot as plt

#Loading saved model architecture
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import ChebConv
from torch_geometric.nn import GATConv
from torch_geometric.nn import TransformerConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn.pool import global_max_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        # self.conv1 = GCNConv(6, 512, improved = True)
        # self.conv2 = GCNConv(512, 256, improved = True)

        # self.conv1 = ChebConv(6, 512, K = 2)
        # self.conv2 = ChebConv(512,256, K = 2)

        self.conv1 = ChebConv(6, 256, K = 3)
        self.conv2 = ChebConv(256,256, K = 4)
        self.conv3 = ChebConv(256, 256, K = 5)
        # self.fc1 = Linear(256, 512)
        # self.fc2 = Linear(512, 256)

        # self.conv1 = GATConv(6, 256, heads = 4)
        # self.conv2 = GATConv(256*4,64, heads = 1, concat=False)

        # self.conv1 = TransformerConv(6, 200, heads = 3)
        # self.conv2 = TransformerConv(200*3,256, heads = 1, concat=False)

        self.lin = Linear(256, 24)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()

        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        # = global_max_pool(x, batch)  # [batch_size, hidden_channels]

        # x = self.fc1(x)
        # x = x.relu()
        # x = self.fc2(x)
        # x = x.relu()

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.2, training=self.training) #0.5
        x = self.lin(x)

        return x

model = GCN(hidden_channels=464)
print(model)

# print(os.getcwd()+ "/model.pth")
# model_save_location = os.getcwd()+ "/model.pth"
print(os.getcwd()+ "/model_34.pt")
model_save_location = os.getcwd()+ "/model_34.pt"
model.load_state_dict(torch.load(model_save_location, map_location=torch.device('cpu')))
print(model.conv3)
print(model.state_dict())

import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import os
import torch
import networkx as nx
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
print(os.getcwd())

# +
import time
#Extract Training Data
#Loads IEEE34 Bus Simulation Data

path = os.getcwd()

numpy_data = np.load(path+"/new_data.npy",allow_pickle = True)
numpy_data = np.squeeze(numpy_data)
print(numpy_data)
print(numpy_data.shape)
print(numpy_data.size)

value_data = np.load(path+"/new_labels.npy",allow_pickle = True)
print(value_data)
print(value_data.shape)
print(value_data.size)


NodeIndex = np.load(path+"/NodeIndex.npy",allow_pickle = True)
print(NodeIndex)
print(NodeIndex.shape)
print(NodeIndex.size)

AdjacencyMatrix = np.load(path+"/AdjacencyMatrix.npy",allow_pickle = True)
print(AdjacencyMatrix)
print(AdjacencyMatrix.shape)
print(AdjacencyMatrix.size)

# +
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import NormalizeFeatures
from torch.nn.functional import normalize

encoder = ['SourceBus', '800', '802', '806', '808', '810', '812', '814', '814r', '850',
'816', '818', '824', '820', '822', '826', '828', '830', '854', '832', '858',
'834', '860', '842', '836', '840', '862', '844', '846', '848', '852r', '888', '856', '852', '864', '838', '890']
print(encoder)
print(len(encoder))
research_paper_decoder = [0,0,1,2,3,4,5,6,6,6,6,7,8,9,10,11,8,12,12,13,14,15,21,15,16,16,16,17,18,18,13,13,19,13,20,22,23]

FaultLocationLabels = value_data[:,3]

for n in range(len(FaultLocationLabels)):
    FaultLocationLabels[n]=research_paper_decoder[encoder.index(str(FaultLocationLabels[n]))]

y = FaultLocationLabels.astype("int64")
y = torch.from_numpy(y)
x = numpy_data.astype("float32")
x = torch.from_numpy(x)

NodeIndex = NodeIndex.astype("int64")
NodeIndex = NodeIndex.T
NodeIndex = torch.from_numpy(NodeIndex)

print(y)
print(y.dtype)
print(x)
print(x.dtype)
print(NodeIndex)
print(NodeIndex.dtype)
#
print(x[0])
print(normalize(x[0]))
#
result_translator = np.unique(FaultLocationLabels.astype("int64")).tolist()
print()
total_data_list = []
for n in range(len(x)):
    #print(x[n])
    #print(y[n])
    DataObject = Data(x = x[n], edge_index = NodeIndex, y = y[n], is_undirected = True) #Testing with non-normalized data
    #DataObject = Data(x = x[n], edge_index = NodeIndex, y = y[n], is_undirected = True)
    DataObject.is_undirected = True
    total_data_list.append(DataObject)
#print('Y'*888)
#print(total_data_list[0].x)

print()
#print(f'Dataset: {total_data_list}:')
print('===================')
print(f'Number of graphs: {len(total_data_list)}')
print(f'Number of features: {total_data_list[0].num_features}')
#print(f'Number of classes: {total_data_list[0].num_classes}')

data = total_data_list[0]  # Get the first graph object.
#print(data)
#print(data.y)

print()
print(NormalizeFeatures(data.x))
print(data.x)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

from random import shuffle
torch.manual_seed(12345)
#total_data_list = total_data_list.shuffle()
shuffle(total_data_list)

train_dataset = total_data_list[:14640] #9150 is half
test_dataset = total_data_list[14640:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')


print(test_dataset[0].x)
print(test_dataset[0])
#print(train_dataset[0].x)

for sample in range(len(test_dataset)):
    # noise = np.random.normal(1,0.03, size = (37,6)) #0.09 #Uncomment for noise
    # noise = noise.astype("float32")
    # noise = torch.from_numpy(noise)
    # test_dataset[sample].x = noise*test_dataset[sample].x
    test_dataset[sample].x = normalize(test_dataset[sample].x)
    #print(sample)

print('Y'*888)
#print(train_dataset[0].x)
print(test_dataset[0].x)
print(test_dataset[0])
##noise = np.random.normal(1,0.09, size = (16,6))
##noise = noise.astype("float32")
##noise = torch.from_numpy(noise)
print('=============================================================')
##
print(train_dataset[0].x)
print(train_dataset[0])
#print(train_dataset[0].x)

for sample in range(len(train_dataset)):
#    noise = np.random.normal(1,0.09, size = (16,6))
#    #    noise = np.random.normal(1,0.09, size = (1,96))
#    noise = noise.astype("float32")
#    noise = torch.from_numpy(noise)
##    print(noise)
##    X_test[sample] = X_test[sample]*noise[0]
#    train_dataset[sample].x = noise*train_dataset[sample].x
    train_dataset[sample].x = normalize(train_dataset[sample].x)
    #print(sample)

print('Y'*888)
#print(train_dataset[0].x)
print(train_dataset[0].x)
print(train_dataset[0])
##print(noise*test_dataset[0].x)
# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

print("$"*200)

#################################################################################
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import torch_geometric.transforms as T
from torch_geometric.datasets import ExplainerDataset
from torch_geometric.datasets.graph_generator import BAGraph
from torch_geometric.explain import Explainer, GNNExplainer, GraphMaskExplainer
from torch_geometric.explain import DummyExplainer, PGExplainer
from torch_geometric.nn import GCN
from torch_geometric.utils import k_hop_subgraph


##dataset = ExplainerDataset(
##    graph_generator=BAGraph(num_nodes=300, num_edges=5),
##    motif_generator='house',
##    num_motifs=80,
##    transform=T.Constant(),
##)
#data = test_dataset[0]
#print(data)

from typing import Tuple

import torch
from torch import Tensor

from torch_geometric.explain import Explainer, Explanation
from torch_geometric.explain.config import ExplanationType, ModelMode


pos_fidelity_list = []
neg_fidelity_list = []
characterization_list = []
#topk_list = [1,5,10,15]
topk_list = [10]
# topk_list = [1,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80]
for topk in topk_list:
    explainer = Explainer(
        model=model,
    #    algorithm=GNNExplainer(epochs=200),
    #    algorithm=PGExplainer(epochs=2),
        # algorithm=GraphMaskExplainer(num_layers = 3, epochs=200),
       algorithm=DummyExplainer(),
        explanation_type='model',
        node_mask_type='attributes',
        edge_mask_type='object',
        model_config=dict(
            mode='multiclass_classification',
            task_level='graph',
            return_type='log_probs',
        ),
        threshold_config=dict(
#            threshold_type='topk',
            threshold_type='topk_hard',
            value =topk,
        ),
    )
    count = 0
    explain_y_hat_list = []
    complement_y_hat_list = []
    y_list = []
    y_hat_list = []

    for batch in test_loader:
        count+=1
        Edge_Index = batch.edge_index[:,:72]
        Batch = batch.batch[:37]
        label1 = batch.y[0].numpy().tolist()#
        input = batch.x[0:37]#

        #Uncomment lines below if running the PGExplainer (only supports phenomenon-based explanation)
    #    for epoch in range(2):
    #        loss = explainer.algorithm.train(epoch=epoch, model=model, x = input, edge_index=Edge_Index, batch = Batch)

        explanation = explainer(x = input, edge_index = Edge_Index, batch = Batch)#, target = label1
        y_hat = model(input, Edge_Index, Batch).argmax(dim=1).item()
    #path = 'subgraph.pdf'
    #explanation.visualize_graph(path)
        node_mask = explanation.get('node_mask')
        edge_mask = explanation.get('edge_mask')

        #Two lines below focus on converting the soft masks to hard masks
     #   node_mask = torch.tensor((node_mask-node_mask.mean())>0, dtype=torch.float32)
    #    edge_mask = torch.tensor((edge_mask-edge_mask.mean())>0, dtype=torch.float32)

#        print(node_mask)
#        print(edge_mask)
    #    print("7"*88)
        kwargs = {key: explanation[key] for key in explanation._model_args}
        print(node_mask.dtype)
        print(edge_mask.dtype)
    #    y = explanation.target
        y = label1
        explain_y_hat = explainer.get_masked_prediction(
                explanation.x,
                explanation.edge_index,
                node_mask,
                edge_mask,
                **kwargs,
            )
        explain_y_hat = explainer.get_target(explain_y_hat)
        complement_y_hat = explainer.get_masked_prediction(
            explanation.x,
            explanation.edge_index,
            1. - node_mask if node_mask is not None else None,
            1. - edge_mask if edge_mask is not None else None,
            **kwargs,
        )
        complement_y_hat = explainer.get_target(complement_y_hat)

        explain_y_hat_list.append(explain_y_hat.numpy()[0])
        complement_y_hat_list.append(complement_y_hat.numpy()[0])
        y_list.append(y)
        y_hat_list.append(y_hat)

        print(count)
        print(explain_y_hat.numpy()[0])
        print(complement_y_hat.numpy()[0])
        print(y)
        print(y_hat)
        print("#########")

#        if count > 1:
#            break

    explain_y_hat_list = np.array(explain_y_hat_list)
    complement_y_hat_list = np.array(complement_y_hat_list)
    y_list = np.array(y_list)
    y_hat_list = np.array(y_hat_list)

    print(explain_y_hat_list )
    print(complement_y_hat_list )
    print(y_list )
    print(y_hat_list)
    #.astype('float').mean()

    pos_fidelity = 1. - (complement_y_hat_list == y_hat_list).astype('float').mean()
    neg_fidelity = 1. - (explain_y_hat_list == y_hat_list).astype('float').mean()
    print("Positive Fidelity = ", pos_fidelity) #positive fidelity reflects necessary explanation
    # It's better for positive fidelity to be 1
    print("Negative Fidelity = ", neg_fidelity) #negative fidelity reflects sufficient explanation
    # It's better for negative fidelity to be 0
    CharacterizationScore = 1./((0.5/pos_fidelity)+(0.5/(1-neg_fidelity)))
    print(CharacterizationScore)

    pos_fidelity_list.append(pos_fidelity)
    neg_fidelity_list.append(neg_fidelity)
    characterization_list.append(CharacterizationScore)

print(topk_list)
print(pos_fidelity_list)
print(neg_fidelity_list)
print(characterization_list)

Topk_Results = pd.DataFrame(data = {'TopK' : topk_list,
    'Fidelity+' : pos_fidelity_list,
    'Fidelity-' : neg_fidelity_list,
    'Characterization' : characterization_list})

#Topk_Results.to_csv(path+'/'+str(explainer.algorithm)[:-2]+'_topk_results.csv', index = False)
Topk_Results.to_csv(path+'/'+str(explainer.algorithm)[:-2]+'_topk_hard_results.csv', index = False)
#.attribution_method_class.__name__
print("  "+str(explainer.algorithm)[:-2])
#print(type(explainer.algorithm.attribution_method_class))
print(path+'/'+str(explainer.algorithm)[:-2]+'_topk_results.csv')
print(Topk_Results)

end_time = time.time()
print("Run Time = ", end_time-start_time)

print(test_loader)



# from IPython.display import IFrame



# # Specify the path to your PDF file
# pdf_path = 'subgraph.pdf'

# # Display the PDF inline
# IFrame(pdf_path, width=600, height=300)

# from pdf2image import convert_from_path

# images = convert_from_path("subgraph.pdf")
# images[0]  # first page




#
#from pdf2image import convert_from_path
#from IPython.display import display, Image
#
#images = convert_from_path("subgraph.pdf")
#for i, image in enumerate(images):
#    fname = "image" + str(i) + ".png"
#    image.save(fname, "PNG")
#Image(fname, width=800, height=1500)
#
#print(NodeIndex)
#print(NodeIndex.shape)
#print(NodeIndex.size)


GCN(
  (conv1): ChebConv(6, 256, K=3, normalization=sym)
  (conv2): ChebConv(256, 256, K=4, normalization=sym)
  (conv3): ChebConv(256, 256, K=5, normalization=sym)
  (lin): Linear(in_features=256, out_features=24, bias=True)
)
/content/model_34.pt
ChebConv(256, 256, K=5, normalization=sym)
OrderedDict([('conv1.bias', tensor([ 6.2356e-02, -2.1974e-01, -4.6219e-02, -1.7084e-01, -2.6293e-01,
        -2.6260e-01, -8.5107e-02, -1.3529e-01, -7.8290e-02, -2.3142e-01,
        -3.1671e-01, -1.5598e-01,  8.2232e-02,  1.0987e-01, -2.9260e-01,
        -2.3244e-01, -1.3441e-01, -2.2913e-01, -2.0324e-01, -1.9280e-01,
        -9.8404e-02, -1.4911e-03,  3.1328e-02,  8.7063e-02,  3.2462e-02,
        -8.5882e-02,  1.2128e-01, -3.3120e-02,  2.4983e-02, -6.0169e-02,
        -3.1656e-01, -3.1808e-01, -2.3165e-01,  1.0383e-01, -2.1685e-01,
        -3.6769e-01, -3.2775e-01,  1.6007e-01,  2.7386e-01,  6.3639e-04,
         6.4094e-02,  1.3523e-02, -2.7450e-01, -5.7802e-02, -2.0482e-01,
        -1.1098e-01, -7

<ipython-input-12-39715202c0aa>:96: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_location, map_location=torch.device('cpu')))


Streaming output truncated to the last 5000 lines.
torch.float32
5468
12
13
8
8
#########
torch.float32
torch.float32
5469
12
13
15
15
#########
torch.float32
torch.float32
5470
12
1
16
16
#########
torch.float32
torch.float32
5471
12
1
17
17
#########
torch.float32
torch.float32
5472
12
1
16
16
#########
torch.float32
torch.float32
5473
12
1
13
13
#########
torch.float32
torch.float32
5474
12
1
17
17
#########
torch.float32
torch.float32
5475
12
1
2
2
#########
torch.float32
torch.float32
5476
12
1
14
14
#########
torch.float32
torch.float32
5477
12
1
21
21
#########
torch.float32
torch.float32
5478
12
1
2
2
#########
torch.float32
torch.float32
5479
12
1
13
13
#########
torch.float32
torch.float32
5480
12
1
6
6
#########
torch.float32
torch.float32
5481
12
1
2
2
#########
torch.float32
torch.float32
5482
12
1
16
16
#########
torch.float32
torch.float32
5483
12
1
14
14
#########
torch.float32
torch.float32
5484
12
1
15
15
#########
torch.float32
torch.float32
5485
12
1
1
1
#########
t